# I. 데이터 프레임 변환

In [1]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # warning 메시지 숨기기

import tensorflow as tf
print("Is GPU available:", tf.test.is_gpu_available())

import tensorflow as tf
print('tf.__version__:',tf.__version__)

2025-03-06 02:29:42.888929: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 02:29:42.940153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 02:29:44.528421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: True
tf.__version__: 2.13.0


2025-03-06 02:29:46.805611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 02:29:46.836996: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 02:29:46.838534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
tf.config.set_visible_devices([], 'GPU')

2025-03-06 02:29:56.047687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 02:29:56.049476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 02:29:56.050951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## 1. 통합 데이터 프레임 생성

In [4]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [6]:
import os
import json
import pandas as pd

def load_and_flatten_json(root_dir):
    data = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.json'):
                filepath = os.path.join(subdir, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        json_data = json.load(f)
                        info = json_data.get('info', {})
                        license_info = json_data.get('license', {})
                        audio = json_data.get('audio', {})
                        environment = json_data.get('environment', {})
                        annotations = json_data.get('annotations', [])

                        area_start = None
                        area_end = None
                        category_01 = None
                        category_02 = None
                        category_03 = None
                        labelName = None
                        soundQuality = None
                        decibel = None
                        subCategory = None

                        if annotations and isinstance(annotations, list) and len(annotations) > 0:
                            annotation = annotations[0]
                            if isinstance(annotation, dict):
                                area = annotation.get('area', {})
                                area_start = area.get('start')
                                area_end = area.get('end')
                                categories = annotation.get('categories', {})
                                category_01 = categories.get('category_01')
                                category_02 = categories.get('category_02')
                                category_03 = categories.get('category_03')
                                labelName = annotation.get('labelName')
                                soundQuality = annotation.get('soundQuality')
                                decibel = annotation.get('decibel')
                                subCategory = annotation.get('subCategory')

                        row = {
                            'area_start': area_start,
                            'area_end': area_end,
                            'category_01': category_01,
                            'category_02': category_02,
                            'category_03': category_03,
                            'decibel': decibel,
                            'labelName': labelName,
                            'soundQuality': soundQuality,
                            'subCategory': subCategory,
                            'bitRate': audio.get('bitRate'),
                            'duration': audio.get('duration'),
                            'fileFormat': audio.get('fileFormat'),
                            'fileName': audio.get('fileName'),
                            'fileSize': audio.get('fileSize'),
                            'recodingType': audio.get('recodingType'),
                            'sampleRate': audio.get('sampleRate'),
                            'acqDevice': environment.get('acqDevice'),
                            'acqMethod': environment.get('acqMethod'),
                            'acqType': environment.get('acqType'),
                            'areaUse': environment.get('areaUse'),
                            'dayNight': environment.get('dayNight'),
                            'direction': environment.get('direction'),
                            'distance': environment.get('distance'),
                            'district': environment.get('district'),
                            'latitude': environment.get('gps', {}).get('latitude'),
                            'longitude': environment.get('gps', {}).get('longitude'),
                            'micClass': environment.get('micClass'),
                            'obstacle': environment.get('obstacle'),
                            'place': environment.get('place'),
                            'recordingTime': environment.get('recordingTime'),
                            'urban': environment.get('urban'),
                            'weather': environment.get('weather'),
                            'contributor': info.get('contributor'),
                            'dateCreated': info.get('dateCreated'),
                            'description': info.get('description'),
                            'uri': info.get('uri'),
                            'version': info.get('version'),
                            'year': info.get('year'),
                            'license_name': license_info.get('name'),
                            'license_url': license_info.get('url'),
                        }
                        data.append(row)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON file: {filepath} - {e}")
    return pd.DataFrame(data)

# JSON 파일 로드 및 평탄화
root_dir_2 = '/home/ubuntu/data/etc_noise_data_test_final'
etc_json_df = load_and_flatten_json(root_dir_2)

print("\n기타 소음 JSON 데이터프레임:")
print(etc_json_df.head())
print(etc_json_df.info())


기타 소음 JSON 데이터프레임:
   area_start  area_end category_01 category_02 category_03  decibel  \
0         2.0     9.430       사업장소음         공사장         발전기       85   
1         2.0    48.958       사업장소음         공사장         발전기      102   
2         2.0    12.402       사업장소음         공사장         발전기       99   
3         2.0     7.084       사업장소음         공사장         발전기       71   
4         2.0    10.000       사업장소음         공사장         발전기       69   

           labelName soundQuality subCategory  bitRate  ...  urban weather  \
0  9.공사장_40349_1.wav           정상       소형발전기  705kbps  ...  서울특별시      맑음   
1  9.공사장_69400_1.wav           정상       대형발전기  768kbps  ...    창원시      맑음   
2  9.공사장_40301_1.wav           정상       소형발전기  705kbps  ...  서울특별시      맑음   
3  9.공사장_34532_1.wav           정상       소형발전기  705kbps  ...  서울특별시      맑음   
4  9.공사장_40176_1.wav          노이즈       소형발전기  705kbps  ...  서울특별시      맑음   

  contributor  dateCreated description                      uri version  year 

In [8]:
import librosa
import numpy as np
import os
import pandas as pd
import re
import json

# 한글-영어 매칭 딕셔너리 보완
kor_eng_dict = {
    "자동차": "car",
    "이륜자동차": "motorcycle",
    "경적": "horn",
    "사이렌": "siren",
    "주행음": "driving_sound",
    # 필요한 모든 매핑 추가
}

# 파일명 정규화 함수 수정
def normalize_filename(filename):
    # 확장자 제거
    name, ext = os.path.splitext(filename)

    # 대소문자 변환 및 공백 제거
    name = name.lower().replace(' ', '')

    # 한글-영어 변환 적용
    for kor, eng in kor_eng_dict.items():
        name = name.replace(kor, eng)

    # 정규식으로 고유 ID 추출
    match = re.match(r".*?_(\d+)(?:_\d+)?$", name)
    if match:
        file_id = match.group(1)
    else:
        file_id = None

    return name, ext, file_id

# 매칭 검사 함수 수정
def find_mismatched_files(directory):
    wav_files = {}
    json_files = {}

    # 모든 폴더 순회하며 파일 수집
    for root, _, files in os.walk(directory):
        for file in files:
            name, ext, file_id = normalize_filename(file)
            if file_id is None:
                continue  # ID가 없는 파일은 패스
            file_path = os.path.join(root, file)
            if ext == ".wav":
                wav_files[file_id] = file_path
            elif ext == ".json":
                json_files[file_id] = file_path

    # 매칭되지 않은 파일 찾기
    mismatched_wav = set(wav_files.keys()) - set(json_files.keys())
    mismatched_json = set(json_files.keys()) - set(wav_files.keys())

    return mismatched_wav, mismatched_json, wav_files, json_files

def extract_mfcc(wav_path):
    """
    주어진 WAV 파일 경로에서 MFCC 특징을 추출합니다.
    """
    try:
        audio, sr = librosa.load(wav_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=50)
        mfcc_mean = np.mean(mfcc, axis=1).tolist()
        return mfcc_mean
    except Exception as e:
        print(f"Error processing WAV file: {wav_path} - {e}")
        return None

def load_wav_and_extract_mfcc(root_dir, df_json):
    """
    주어진 디렉토리의 모든 WAV 파일을 로드하고 MFCC 특징을 추출하여 DataFrame에 추가합니다.
    """
    mfcc_data = []
    for index, row in df_json.iterrows():
        wav_filename = row['fileName']
        wav_path = None

        # JSON 파일 이름 정규화
        json_name, _, json_id = normalize_filename(wav_filename)

        # 모든 하위 디렉토리에서 WAV 파일 검색
        for subdir, dirs, files in os.walk(root_dir):
            for file in files:
                wav_name, _, wav_id = normalize_filename(file)
                if json_id == wav_id and file.endswith('_1.wav'):  # ID가 일치하고 _1로 끝나는 파일
                    wav_path = os.path.join(subdir, file)
                    break

        if wav_path:
            mfcc_features = extract_mfcc(wav_path)
            mfcc_data.append(mfcc_features)
        else:
            print(f"WAV file not found for: {wav_filename}, looking for ID: {json_id}")
            mfcc_data.append(None)

    mfcc_columns = [f'mfcc_{i+1}' for i in range(50)]
    df_mfcc = pd.DataFrame(mfcc_data, columns=mfcc_columns)
    return pd.concat([df_json, df_mfcc], axis=1)

# JSON 파일 로드 및 평탄화
root_dir_2 = '/home/ubuntu/data/etc_noise_data_test_final'
etc_json_df = load_and_flatten_json(root_dir_2)

# WAV 파일 로드 및 MFCC 추출
df_final_2 = load_wav_and_extract_mfcc(root_dir_2, etc_json_df)
print(df_final_2.head())

   area_start  area_end category_01 category_02 category_03  decibel  \
0         2.0     9.430       사업장소음         공사장         발전기       85   
1         2.0    48.958       사업장소음         공사장         발전기      102   
2         2.0    12.402       사업장소음         공사장         발전기       99   
3         2.0     7.084       사업장소음         공사장         발전기       71   
4         2.0    10.000       사업장소음         공사장         발전기       69   

           labelName soundQuality subCategory  bitRate  ...   mfcc_41  \
0  9.공사장_40349_1.wav           정상       소형발전기  705kbps  ... -2.654079   
1  9.공사장_69400_1.wav           정상       대형발전기  768kbps  ... -1.609383   
2  9.공사장_40301_1.wav           정상       소형발전기  705kbps  ... -3.747083   
3  9.공사장_34532_1.wav           정상       소형발전기  705kbps  ...  0.331332   
4  9.공사장_40176_1.wav          노이즈       소형발전기  705kbps  ... -0.066167   

    mfcc_42   mfcc_43   mfcc_44   mfcc_45   mfcc_46   mfcc_47   mfcc_48  \
0 -0.558064  1.090053  0.112485  0.559785 -0.939016 -

## 2. 열 순서 조정

In [9]:
# 기존 데이터프레임 일부 불러오기

import config 
from config import DB_CONFIG
import pymysql  
 
# MySQL 연결
conn = pymysql.connect(
     host=DB_CONFIG['host'],
     user=DB_CONFIG['user'],
     password=DB_CONFIG['password'],
     database=DB_CONFIG['database'],
     port=DB_CONFIG['port']
 )
 
car_siren = f"SELECT * FROM merge50_car_siren"
 
car_siren_df =  pd.read_sql(car_siren, conn)
 
conn.close()

/tmp/ipykernel_4099/793792915.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_siren_df =  pd.read_sql(car_siren, conn)


## 3. 최종 데이터프레임 생성

In [10]:
# df_combined의 열 목록 가져오기
combined_cols = car_siren_df.columns.tolist()

# df_final_2에서 df_combined에 있는 열만 선택
final_cols = [col for col in combined_cols if col in df_final_2.columns]
df_final_2 = df_final_2[final_cols]

# df_final_2의 열 순서를 df_combined의 열 순서에 맞춰 변경
df_final_2 = df_final_2[combined_cols[:len(final_cols)]]

print(df_final_2)

             fileName          labelName  area_start  area_end category_01  \
0     9.공사장_40349.wav  9.공사장_40349_1.wav        2.00     9.430       사업장소음   
1     9.공사장_69400.wav  9.공사장_69400_1.wav        2.00    48.958       사업장소음   
2     9.공사장_40301.wav  9.공사장_40301_1.wav        2.00    12.402       사업장소음   
3     9.공사장_34532.wav  9.공사장_34532_1.wav        2.00     7.084       사업장소음   
4     9.공사장_40176.wav  9.공사장_40176_1.wav        2.00    10.000       사업장소음   
...               ...                ...         ...       ...         ...   
2899   7.동물_21233.wav   7.동물_21233_1.wav        2.00    22.464        생활소음   
2900   7.동물_49714.wav   7.동물_49714_1.wav        2.00    12.311        생활소음   
2901   7.동물_50077.wav   7.동물_50077_1.wav        2.00    26.916        생활소음   
2902   7.동물_49929.wav   7.동물_49929_1.wav        2.00    29.847        생활소음   
2903   7.동물_93791.wav   7.동물_93791_1.wav        2.55     5.780        생활소음   

     category_02 category_03  decibel soundQuality subCategory 

## 4. 기타소음 분류 칼럼 추가

In [11]:
# df_result에서 실제 라벨을 기준으로 정상 데이터와 기타 소음 라벨링
valid_labels = ['차량경적', '차량주행음', '이륜차경적', '이륜차주행음', '차량사이렌']  # 정상 라벨

# 'category_03' 컬럼이 정상 라벨이면 그대로 두고, 그렇지 않으면 '기타소음'으로 설정
df_final_2['ood_label'] = df_final_2['category_03'].apply(lambda x: x if x in valid_labels else '기타소음')

In [13]:
print(df_final_2['category_03'].value_counts())

category_03
이륜차경적     519
이륜차주행음    498
차량경적      399
고양이       274
차량사이렌     249
차량주행음     227
개         217
공구        172
항타기       133
발전기       123
콘크리트펌프     93
Name: count, dtype: int64


In [14]:
print(df_final_2['ood_label'].value_counts())

ood_label
기타소음      1012
이륜차경적      519
이륜차주행음     498
차량경적       399
차량사이렌      249
차량주행음      227
Name: count, dtype: int64


## 5. csv 파일로 저장

In [15]:
import pandas as pd

# 현재 데이터프레임 원본을 CSV 파일로 저장
df_final_2.to_csv('final_test_data_original.csv', index=False)

In [17]:
import pandas as pd

# category_03의 각 레이블별로 90개씩 랜덤 샘플링
sampled_data_category = df_final_2.groupby('category_03').apply(lambda x: x.sample(min(len(x), 90), random_state=42))

# 샘플링된 데이터를 CSV 파일로 저장
sampled_data_category.to_csv('final_test_sampled_90_category_03.csv', index=False)

In [21]:
print(sampled_data_category['category_03'].value_counts())

category_03
개         90
고양이       90
공구        90
발전기       90
이륜차경적     90
이륜차주행음    90
차량경적      90
차량사이렌     90
차량주행음     90
콘크리트펌프    90
항타기       90
Name: count, dtype: int64


In [22]:
print(sampled_data_category['ood_label'].value_counts())

ood_label
기타소음      540
이륜차경적      90
이륜차주행음     90
차량경적       90
차량사이렌      90
차량주행음      90
Name: count, dtype: int64


In [18]:
import pandas as pd

def sample_ood_with_category_balance(df, n=220, random_state=42):
    """ood_label을 기준으로 샘플링하고, '기타소음'의 경우 category_03 분포를 균등하게 조정합니다."""

    sampled_data = []
    for ood_label, group in df.groupby('ood_label'):
        if ood_label == '기타소음':
            # '기타소음'의 경우 category_03 분포를 균등하게 조정하여 총 1500개 샘플링
            category_counts = group['category_03'].value_counts()
            num_categories = len(category_counts)
            samples_per_category = n // num_categories  # 각 카테고리별 샘플 수
            remainder = n % num_categories  # 나머지 샘플 수

            samples = []
            for category, count in category_counts.items():
                sample_count = min(count, samples_per_category)
                samples.append(group[group['category_03'] == category].sample(sample_count, random_state=random_state))

            # 나머지 샘플을 랜덤하게 추가
            remaining_samples = pd.concat(samples)
            if remainder > 0:
                remaining_samples = pd.concat([remaining_samples, group[~group.index.isin(remaining_samples.index)].sample(remainder, random_state=random_state)])

            sampled_data.append(remaining_samples)
        else:
            # 다른 ood_label은 기존 방식대로 샘플링
            sampled_data.append(group.sample(min(len(group), n), random_state=random_state))

    return pd.concat(sampled_data)

# 수정된 함수를 사용하여 샘플링
sampled_data_ood = sample_ood_with_category_balance(df_final_2)

# 샘플링된 데이터를 CSV 파일로 저장
sampled_data_ood.to_csv('final_test_data_sampled_220_ood_label.csv', index=False)

In [19]:
print(sampled_data_ood['category_03'].value_counts())

category_03
이륜차경적     220
이륜차주행음    220
차량경적      220
차량사이렌     220
차량주행음     220
개          38
고양이        37
발전기        37
공구         36
항타기        36
콘크리트펌프     36
Name: count, dtype: int64


In [20]:
print(sampled_data_ood['ood_label'].value_counts())

ood_label
기타소음      220
이륜차경적     220
이륜차주행음    220
차량경적      220
차량사이렌     220
차량주행음     220
Name: count, dtype: int64
